# Caffe的Python接口学习(2)：生成solver文件

Caffe在训练的时候，需要一些参数设置，我们一般将这些参数设置在一个叫solver.prototxt的文件里面，如下：

```
base_lr: 0.001
display: 782
gamma: 0.1
lr_policy: "step"
max_iter: 78200
momentum: 0.9
snapshot: 7820
snapshot_prefix: "snapshot"
solver_mode: GPU
solver_type: SGD
stepsize: 26067
test_interval: 782
test_iter: 313
test_net: "/root/Repository/caffe/examples/mine/python/val.prototxt"
train_net: "/root/Repository/caffe/examples/mine/python/train.prototxt"
weight_decay: 0.0005
```

有一些参数需要计算，也不是乱设置。

假设我们有50000个样本，batch_size为64，即每批次处理64个样本，那么需要迭代$50000 \div 64 = 782$次才处理完一次全部的样本。我们把处理完一次所有的样本，称之为一代，即epoch。所以，这里的test_interval设置为782，即处理完一次所有的训练数据后，才去进行测试。如果我们想训练100代，则需要设置max_iter为78200。

同理，如果有10000个测试样本，batch_size设为32，那么需要迭代$10000 \div 32 = 313$次才完整地测试完一次，所以设置test_iter为2。

学习率变化率我们设置为随着迭代次数的增加，慢慢变低。总共迭代78200次，我们将变化lr_rate三次，所以stepsize设置为$78200 \div 3 = 26067$，即每迭代26067次，我们就降低一次学习率。

下面是生成solver文件的Python代码，比较简单：

``` python
# -*- coding: utf-8 -*-

path = '/root/Repository/caffe/examples/mine/python/'
solver_file = path + 'solver.prototxt' # solver文件保存位置

sp = {}
sp['train_net'] = '"' + path + 'train.prototxt"' # 训练配置文件
sp['test_net'] = '"' + path + 'val.prototxt"' # 测试配置文件
sp['test_iter'] = '313' # 测试迭代次数
sp['test_interval'] = '782' # 测试间隔
sp['base_lr'] = '0.001' # 基础学习率
sp['display'] = '782' # 屏幕日志显示间隔
sp['max_iter'] = '78200' # 最大迭代次数
sp['lr_policy'] = '"step"' # 学习率变化规律
sp['gamma'] = '0.1' # 学习率变化指数
sp['momentum'] = '0.9' # 动量
sp['weight_decay'] = '0.0005' # 权值衰减
sp['stepsize'] = '26067' # 学习率变化频率
sp['snapshot'] = '7820' # 保存model间隔
sp['snapshot_prefix'] = '"snapshot"' # 保存的model前缀
sp['solver_mode'] = 'GPU' # 使用GPU
sp['solver_type'] = 'SGD' # 优化算法

def write_solver():
    # 写入文件
    with open(solver_file, 'w') as f:
        for key, value in sorted(sp.items()):
            if not(type(value) is str):
                raise TypeError('All solver parameters must be strings.')
            f.write('%s: %s\n' % (key, value))

if __name__ == '__main__':
    write_solver()
```

执行上面的文件，我们就会得到一个solver.prototxt文件，有了这个文件，我们下一步就可以进行训练了。

当然，如果你觉得上面这种键值对的字典方式，写起来容易出错，我们也可以使用另外一种比较简便的方法，没有引号，不太容易出错，如下：

``` python
# -*- coding: utf-8 -*-

from caffe.proto import caffe_pb2

s = caffe_pb2.SolverParameter()

path = '/root/Repository/caffe/examples/mine/python/'
solver_file = path + 'solver.prototxt'

s.train_net = path + 'train.prototxt'
s.test_net.append(path + 'val.prototxt')
s.test_interval = 782
s.test_iter.append(313)
s.max_iter = 78200

s.base_lr = 0.001
s.momentum = 0.9
s.weight_decay = 5e-4
s.lr_policy = 'step'
s.stepsize = 26067
s.gamma = 0.1
s.display = 782
s.snapshot = 7820
s.snapshot_prefix = 'snapshot'
s.type = "SGD"
s.solver_mode = caffe_pb2.SolverParameter.GPU

with open(solver_file, 'w') as f:
    f.write(str(s))
```